# Eval

In [1]:
; eval
(eval '(+ 1 2 3))
(eval '(format t "Hello"))

6

NIL

Hello

In [1]:
(defun our-toplevel ()
  (do ()
      (nil)
    (format t "~%> ")
    (let ((in (read)))
      (format t "~A" in)
      (print (eval in)))))

(our-toplevel)

OUR-TOPLEVEL


> 1
1 
> (+ 1 2)
3 
> 

interrupt: Execution interrupted

(EXIT)

In [8]:
#'max

#<FUNCTION MAX>

In [16]:
; coerce
(coerce '(lambda (x) x) 'function)
; ((coerce '(lambda (x) x) 'function) 10)
(let ((f (coerce '(lambda (x) x) 'function)))
  (apply f '(10))
  (funcall f 10))

#<FUNCTION (LAMBDA (X)
             :IN
             "C:/Users/zhouj/AppData/Local/Temp/0E450363026B4963FA0A56C071B7F2FD-400920317.lisp") {10010272AB}>

10

In [17]:
; compile
(compile nil '(lambda (x) (+ x 2)))
(let ((f (compile nil '(lambda (x) (+ x 2)))))
  (apply f '(10))
  (funcall f 10))

#<FUNCTION (LAMBDA (X)
             :IN
             "C:/Users/zhouj/AppData/Local/Temp/0E450363026B4963FA0A56C071B7F2FD-3493497194.lisp") {10010264AB}>

NIL

NIL

12

# Macros

In [18]:
(defmacro nil! (x)
  (list 'setf x nil))

(nil! x)
x

NIL!

NIL

NIL

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X


In [20]:
(macroexpand-1 '(nil! x))

(macroexpand '(nil! x))

(SETF X NIL)

T

(SETQ X NIL)

T

# Backquote

In [22]:
`(a b c)
'(a b c)

(let ((a 1)
      (b 2))
  `(a is ,a and b is ,b))

(A B C)

(A B C)

(A IS 1 AND B IS 2)

In [23]:
(defmacro nil! (x)
  `(setf ,x nil))

NIL!

SB-KERNEL:REDEFINITION-WITH-DEFMACRO: redefining COMMON-LISP-USER::NIL! in DEFMACRO


In [27]:
(setf lst '(a b c))
`(lst is ,lst)
`(its elements are ,@lst)

(A B C)

(LST IS (A B C))

(ITS ELEMENTS ARE A B C)

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::LST
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::LST
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::LST


In [1]:
(defmacro while (test &rest body)
  `(do ()
       ((not ,test))
     ,@body))

(let ((x 0))
  (while (< x 10)
         (princ x)
         (incf x)))

WHILE

NIL

0123456789

# Example: Quicksort

In [5]:
(defun quicksort (vec l r)
  (let ((i l)
        (j r)
        (p (svref vec (round (+ l r) 2)))) ; 1
    (while (<= i j) ; 2
      (while (< (svref vec i) p) (incf i))
      (while (> (svref vec j) p) (decf j))
      (when (<= i j)
            (rotatef (svref vec i) (svref vec j))
            (incf i)
            (decf j)))
    (if (> (- j l) 1) (quicksort vec l j)); 3
    (if (> (- r i) 1) (quicksort vec i r)))
  vec)

(let ((v #(6 9 8 1 5 2 3 7 4)))
  (quicksort v 0 (- (length v) 1)))

QUICKSORT

#(1 2 3 4 5 6 8 7 9)

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::QUICKSORT in DEFUN


# Macro Design

In [ ]:
; NOTE: unintended variable capture, multiple evaluation
(defmacro ntimes (n &rest body)
  (let ((g (gensym))
        (h (gensym)))
    `(let ((,h ,n))
       (do ((,g 0 (+ ,g 1)))
           ((>= ,g ,h))
         ,@body))))

(let ((x 10))
  (ntimes 5
          (setf x (+ x 1)))
  x)

NTIMES

15

SB-KERNEL:REDEFINITION-WITH-DEFMACRO: redefining COMMON-LISP-USER::NTIMES in DEFMACRO


In [ ]:
(macroexpand-1 '(cond (a b)
                      (c d e)
                      (t f)))
; pprint
(pprint (macroexpand-1 '(cond (a b)
                              (c d e)
                              (t f))))

(IF A
    B
    (IF C
        (PROGN D E)
        (THE T F)))

T


(IF A
    B
    (IF C
        (PROGN D E)
        (THE T F)))

# Generialized Reference

In [11]:
; define-modify-macro

(defmacro cah (lst)
  `(car ,lst))

(let ((x (list 'a 'b 'c)))
  (setf (cah x) 44)
  x)

CAH

(44 B C)

In [19]:
(defmacro incf-wrong (x &optional (y 1))
  `(setf ,x (+ ,x ,y)))


(let ((lst nil))
  (incf (car (push 1 lst)))
  lst)
(let ((lst nil))
  (incf-wrong (car (push 1 lst)))
  lst)

(macroexpand-1 '(incf-wrong (car (push 1 lst))))

INCF-WRONG

(2)

(1 2)

(SETF (CAR (PUSH 1 LST)) (+ (CAR (PUSH 1 LST)) 1))

T

SB-KERNEL:REDEFINITION-WITH-DEFMACRO: redefining COMMON-LISP-USER::INCF-WRONG in DEFMACRO


In [20]:
(define-modify-macro our-incf (&optional (y 1)) +)
(let ((lst nil))
  (our-incf (car (push 1 lst)))
  lst)

(macroexpand-1 '(our-incf (car (push 1 lst))))

OUR-INCF

(2)

(LET* ((#:G241 (PUSH 1 LST)))
  (SB-KERNEL:%RPLACA #:G241 (+ (CAR #:G241) 1)))

T

In [21]:
; append to the end of a list
(define-modify-macro append1f (val)
                     (lambda (lst val) (append lst (list val))))

(let ((lst '(a b c)))
  (append1f lst 'd)
  lst)

APPEND1F

(A B C D)

# Example: Macro Utilities

In [ ]:
; TODO: macror utilities
; for
; in
; random-choice
; avg
; with-gensyms
; aif

# On Lisp

In [ ]:
; John Foderaro called Lisp 'a programmable programming language'.